In [ ]:
# Se importan todas las librerías necesarias para trabajar en el proyecto
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import missingno as msno
from scipy.stats import shapiro
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,  make_scorer, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import StandardScaler

In [ ]:
def ReadGitHubUrl(url):
  try:
    dataframe = pd.read_csv(url)
    return dataframe
  except Exception as e:
    print(f"Error al leer el archivo CSV desde la URL. Detalles: {e}")
    return None

In [ ]:
# Lectura del dataset desde el repositorio de Github
df1 = ReadGitHubUrl('https://raw.githubusercontent.com/andreuccigermangithub/DataScience-ProyectoFinal2024/main/healthcare-stroke-data.csv')
df1.head()

In [ ]:
# A continuación información estadística del dataset
df1.describe()

In [ ]:
# A continuación información de los tipos de datos de cada columna
df1.info()

In [ ]:
# Gráfico que muestra los datos faltantes en el dataset con missingno
%matplotlib inline
msno.matrix(df1)

Se pueden observar datos faltantes en la columna de BMI (índice de masa corporal), por lo cual se procede a tratar y corregir éste inconveniente en el dataset

In [ ]:
# A continuación se comprueba la normalidad de los valores proporcionados en la columna "BMI" 
# para decidir que método es más conveniente a la hora de reemplezar los valores NAN que contiene.
stat, pvalue = shapiro(df1["bmi"].dropna())

if pvalue > 0.05:
    print("La distribución de los datos es normal.")
else:
    print("La distribución de los datos no es normal.")

In [ ]:
# Al no poseer distribución normal, se opta por reemplezar los datos NAN con la mediana de la columna
mediana = df1["bmi"].median()
df1["bmi"].fillna(mediana, inplace=True)

In [ ]:
# Para comprobar que los datos fueron reemplezados satisfactoriamente, se grafica nuevamente con missingno
%matplotlib inline
msno.matrix(df1)

In [ ]:
# También a través del siguiente comando confirmo la coincidencia en la cantidad de valores no nulos de todas las columnas
df1.info()

In [ ]:
# Boxplots para la verificación de outliers de las columna de "Edad", "Nivel promedio de glucosa", e "Índice de masa corporal".

fig, axs = plt.subplots(1, 3, figsize=(14, 2))

sns.boxplot(x=df1["age"], palette="Reds", ax=axs[0])
sns.boxplot(x=df1["avg_glucose_level"], palette="Blues", ax=axs[1])
sns.boxplot(x=df1["bmi"], palette="Greens", ax=axs[2])

axs[0].set_title("Edad de los pacientes")
axs[1].set_title("Nivel de glucosa promedio")
axs[2].set_title("Índice de masa corporal (BMI)")

plt.tight_layout()
plt.show()

A través de los boxplots, se analizan los valores outliers del dataset, que en éste caso corresponden a Edad de pacientes, Nivel de glucosa promedio e Índice de masa corporal. Para éstos últimos dos mencionados, se confirma la existencia de valores *anormales* dentro del dataset.
En el caso del nivel promedio de glucosa, aparecen valores solamente por encima del valor máximo dentro del boxplot. Y para el caso del BMI, se nota un solo valor por debajo del mínimo, una cantidad considerable inmediatamente luego del valor máximo, y además unos pocos que casi duplican el valor máximo del boxplot.

In [ ]:
# Para corroborar la validez de la información obtenida en el dataset, se procede a verificar si existen valores duplicados dentro del
# dataset que puedan estar alterando los resultados a la hora de manipularlos. Se elige la columna "id" para comprobar si se repite la
# información de algún paciente por error.

# Verificación de valores duplicados en la columna "id" del dataset.
dup_id = df1["id"].duplicated()

# Conteo de la cantidad de True en la columna "dup_id"
cant_dup = dup_id.sum()

# Mensaje
if cant_dup == 0:
    print("No existen datos duplicados en el dataset")
else:
    print(f"Existen {cant_dup} cantidad de datos duplicados")

# **Abstract**
El siguiente trabajo presenta la información médica de un grupo de pacientes, de los cuales una determinada cantidad ha sufrido paro cardíaco al menos una vez. Los datos obtenidos corresponden tanto a su vida personal como así también a su salud en los distintos aspectos. Algunos de estos son estado civil, tipo de empleo, nivel de glucosa, tabaquismo, etc. Con el análisis de los datos proporcionados se pretende determinar que factores combinados contribuyen a que las personas sean más propensas a sufrir un paro cardíaco . De esta forma, se puede ayudar a los médicos a tomar medidas para reducir el riesgo de estos eventos, como recomendar cambios en el estilo de vida o prescribir medicamentos.

**Contexto**

La prepaga de salud *HealthArg* pretende realizar un estudio analítico sobre sus clientes/pacientes que le permita determinar cuales son los valores y variables que aumentan las probabilidades de sufrir un paro cardíaco entre su nómina. De esta forma, con un modelo de predicción adecuado y eficaz, podrán en primer lugar que pacientes corren un mayor peligro de sufrir el paro. Por otro lado, este tipo de predicción contribuiría a reorganizar también su modelo de negocio, analizando que pacientes, pueden en un futuro presentar un mayor gasto en la cobertura de su servicio.

**Modelo predictivo**

Para llevar a cabo este modelo predictivo, se cree que se debe utilizar un modelo supervisado. La razón principal es que se dispone de información sobre si el paciente sufrió un paro cardíaco o no. Esta información se puede utilizar para entrenar el modelo y que aprenda a identificar los patrones que asocian los factores de riesgo con el paro cardíaco.

Se puede utilizar un modelo de regresión lineal para predecir las probabilidades de un paciente de sufrir un paro cardíaco. La regresión lineal se puede utilizar para este tipo de tareas porque asume que la relación entre la variable independiente y la variable dependiente es lineal. Esto significa que la probabilidad de que un paciente sufra un paro cardíaco puede expresarse como una función lineal de los factores de riesgo.

Por ejemplo, se ha demostrado que la edad y el sexo masculino son factores de riesgo independientes para el paro cardíaco. Esto significa que la probabilidad de paro cardíaco aumenta con la edad y es mayor en los hombres que en las mujeres. Estas relaciones pueden modelarse mediante una función lineal.

# **Hipótesis planteada**
A continuación se plantean las hipótesis  y respectivos gráficos para llevar a cabo el estudio del dataset incorporado

# Hipótesis 1:
Un mayor nivel de glucosa en los adultos mayores de 50 años incrementa las probabilidades de padecer un ataque cardíaco.
Para avanzar con ésta afirmación se propone un gráfico tipo scatterplot que muestra la relación existente entre el nivel promedio de glucosa de los pacientes y su edad. Se puede observar que éste nivel se mantiene distribuida de forma homogénea entre 50 y aproximadamnte 110 para todas las edad. Pero la situación cambia al incrementarse la edad ya que para aquellos entre aproximadamente 50 años y 80 hay una gran concentración en los niveles altos de glucosa, desde 175 hasta 250.  

In [ ]:
## Gráfico scatterplot de relación edad/nivel de glucosa

# Obtención de datos
edad = df1['age']
nivel_glucosa_prom = df1['avg_glucose_level']

# Creación del gráfico
plt.scatter(edad, nivel_glucosa_prom)

# Títulos y las etiquetas
plt.title('Relación entre la edad y el nivel de glucosa')
plt.xlabel('Edad')
plt.ylabel('Nivel de glucosa promedio')

# Gráfico
plt.show()

In [ ]:
from matplotlib.pyplot import figtext
hombres = df1.loc[df1["gender"] == "Male"]
mujeres = df1.loc[df1["gender"] == "Female"]

#Crea la figura y los subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Gráfico para mujeres
sns.scatterplot(x=mujeres["age"], y=mujeres["avg_glucose_level"], color="violet", ax=ax1, label="Mujeres")
ax1.set_xlabel("Edad")
ax1.set_ylabel("Nivel Promedio de Glucosa")

# Gráfico para hombres
sns.scatterplot(x=hombres["age"], y=hombres["avg_glucose_level"], color="green", ax=ax2, label="Hombres")
ax2.set_xlabel("Edad")
ax2.set_ylabel("Nivel Promedio de Glucosa")

# Ajusta el espacio entre subplots
plt.subplots_adjust(wspace=0.35)

figtext(0.49, 0.95, 'Relación entre la edad y el nivel de glucosa según género', ha='center', va='top', fontsize=14)

# Muestra el gráfico
plt.show()

# Hipótesis 2:
Aquellos pacientes que han declarado ser fumadores son mas propensos a sufrir un ataque cardíaco que aquellos que dicen haber dejado éste hábito.
En este caso, utilizando un gráfico de barra primero se evalúa a los pacientes respecto a su estado y relación con el tabaquismo. Una gran mayoría jamás ha fumado, mientras que aproximadamente la mitad de ellos fumaba hasta el momento del estudio.

In [ ]:
## Gráfico para los distintos estados de tabaquismo

# Conteo del número de casos para cada categoría
counts = df1["smoking_status"].value_counts()

# Creación del barplot
sns.barplot(x=counts.index, y=counts.values, palette='viridis')

# Adición de títulos y etiquetas
plt.title("Distribución del estado del tabaquismo")
plt.ylabel('Cantidad de pacientes')
plt.xlabel('Estado del tabaquismo')

# Gráfico
plt.show()

# Hipótesis 3:
La edad es un factor fundamental que determina la probabilidad de sufrir un ataque, siendo que a mayor edad de la persona, mayores son las chances. Esto se debe al desmejoramiento de salud de los pacientes al incrementar su edad.
Para analizar la distribución de las edades de aquellos pacientes que han sufrido un ataque cardíaco, se opta por mostrar ésta información con un histograma. Se puede ver que a partir de los 50 años hay un aumento notable respecto a las edades inferiores, el cual siguen aumentando de manera paulatina. Pero realmente hay un incremento muy notorio a partir de los 73/74 años donde ya hay mas de 100 casos, duplicando con creces el rango anterior. Ésto deja en evidencia que a partir de estas edades, las probabilidades de sufrir un ataque cardíaco son mucho mas altas

In [ ]:
## Gráfico tipo histograma para distribución de edades de pacientes que han sufrido para cardíaco

# Creación de un DataFrame de personas que han sufrido para cardíaco
stroke_df = df1[df1["stroke"] == 1]

# Creación de una Series con las edades de las personas que han sufrido ataque cardíaco
stroke_ages = stroke_df["age"]

# Creación del histograma
sns.histplot(stroke_ages, bins=10)

# Adición de títulos y etiquetas
plt.title("Distribución de las edades de personas que han sufrido un ataque")
plt.xlabel("Edad")
plt.ylabel("Cantidad de afectados")

# Gráfico
plt.show()

In [ ]:
# Gráfico tipo histograma para distribución de edades de pacientes que han sufrido para cardíaco

# Creación de un DataFrame de personas que han sufrido para cardíaco
stroke_df = df1[df1["stroke"] == 1]

hombres = stroke_df.loc[stroke_df["gender"] == "Male"]
hombres_edades = hombres["age"]

mujeres = stroke_df.loc[stroke_df["gender"] == "Female"]
mujeres_edades = mujeres["age"]

# Crea la figura y los subplots
fig, (ax2, ax1) = plt.subplots(1, 2, figsize=(10, 4))

# Histograma para mujeres
sns.histplot(data=mujeres_edades, bins=10, color="violet", element="step", ax=ax2)
ax2.set_xlabel("Edad mujeres")
ax2.set_ylabel("Cantidad de pacientes afectadas")

 # Histograma para hombres
sns.histplot(data=hombres_edades, bins=10, color="green", element="step", ax=ax1)
ax1.set_xlabel("Edad hombres")
ax1.set_ylabel("Cantidad de pacientes afectados")

# Ajusta el espacio entre subplots
plt.subplots_adjust(wspace=0.3)

# Agrega leyenda
#plt.legend(loc='upper center', ncol=2)

# Adición de título general
figtext(0.49, 0.95, 'Distribución de las edades de personas que han sufrido un ataque', ha='center', va='top', fontsize=14)

# Muestra el gráfico
plt.show()

# **Elección e implementación del algoritmo de regresión**


In [ ]:
# Se extraen del dataset aquellas columnas que contienen strings que no pueden ser interpretado por el modelo
dfx = df1.copy()
dfx = dfx.drop(columns= ['avg_glucose_level', 'gender', 'ever_married', 'work_type',	'Residence_type', 'smoking_status'])
dfx

In [ ]:
# variable y
dfy = df1[['avg_glucose_level']]
dfy

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(dfx,dfy,test_size=0.3,random_state=2)
# Creación el modelo
lr = LinearRegression()
# Ajuste del modelo con X_train y y_train
lr.fit(X_train,y_train)
# Predicción con X_test
y_pred = lr.predict(X_test)

In [ ]:
print(y_test['avg_glucose_level'].mean())

In [ ]:
# cálculo del error absoluto medio entre el y_test y el valor predicho
print("MAE",mean_absolute_error(y_test,y_pred))
#aquí se compara el valor obtenido MAE con la media de la línea anterior

In [ ]:
# cálculo del error cuadrático medio entre el valor real y y el valor predicho
print("MSE",mean_squared_error(y_test,y_pred))

In [ ]:
# cálculo de la raíz cuadrada error cuadrático medio entre el valor real y y el valor predicho
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
# RMSE logarítmcia
print("RMSE",np.log(np.sqrt(mean_squared_error(y_test,y_pred))))

In [ ]:
# R2 entre y_test y el y predicho
r2 = r2_score(y_test,y_pred)
print(r2)

El valor R2 obtenido da muy cercano a 0 lo cual indica que el modelo propuesto no es eficaz al momento de realizar la predección.

**A continuación se aplica el método PCA para evaluar resultados y verificar su eficacia**

In [ ]:
# Normalización de los datos para poder trabajar mejor con mis variables

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Aplico método de PCA

pca = PCA()
X_train = pca.fit_transform(X_train) #entrena y transforma X_train
X_test = pca.transform(X_test) # transforma X_test

In [ ]:
# Análisis de la varianza
explained_variance = pca.explained_variance_ratio_
explained_variance

In [ ]:
# Para tener un acierto de variavilidad de aproximadamente 90% me quedo con 5
# componentes principales

pca = PCA(n_components=5)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
# Creación del modelo
lr = LinearRegression()
# Ajustar el modelo con X_train y y_train
lr.fit(X_train,y_train)
# Predección con X_test
y_pred = lr.predict(X_test)

In [ ]:
# Cálculo de métricas

# Cálculo del Mean Absolute Error (MAE) entre el valor real y y el valor predicho
mae = mean_absolute_error(y_test,y_pred)
mae = round(mae, 3)
print("_ El Mean Absolute Error (MAE) del modelo es",mae)

# Cálculo del Median Absolute Error (Med AE) entre el valor real y y el valor predicho
med_ae = median_absolute_error(y_test,y_pred)
med_ae = round(med_ae, 3)
print("_ El Median Absolute Error (Med AE) del modelo es", med_ae)

# Cálculo del Relative Absolute Error (RAE)
mean_y_test = np.mean(y_test)
rae = mean_absolute_error(y_test,y_pred)/mean_y_test
rae = round(rae, 3)
print("_ El Relative Absolute Error (RAE) del modelo es", rae)

# Cálculo de la raíz cuadrada error cuadrático medio (RMSE) entre el valor real y y el valor predicho
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
rmse = round(rmse, 3)
print("_ El Root Mean Square Error (RMSE) del modelo es",rmse)

# R2 entre y_test y el y predicho
r2 = r2_score(y_test,y_pred)
r2 = round(r2, 3)
print("_ El coeficiente de determinación (R2) del modelo es ",r2)

**Creación e incorporación de variable sintética al modelo elegido**

In [ ]:
# Se extraen del dataset aquellas columnas que contienen strings que no pueden ser interpretado por el modelo reincor
dfz = df1.copy()
dfz = dfz.drop(columns= ['gender', 'ever_married', 'work_type',	'Residence_type', 'smoking_status'])
dfz

# Creación de variable sintética

# Se opta por multiplicar la edad con el nivel promedio de glucosa (de cada
# paciente) para establecer un valor que permita ver si existe una correlación
# entre ambas variables, ya que se espera que a mayor edad, mayor sea el nivel
# de glusoca del paciente en cuestión. La atención debe centrarse solo en el
# valor de la variable (escalar), ya que de la multiplicación surge una mezcla de unidades
# que no representan ninguna variable en particular o existente.

dfz['var_sint'] = dfz["age"]*dfz["avg_glucose_level"]
dfz

In [ ]:
# Se procede nuevameente con la separación de train y test
X_train,X_test,y_train,y_test = train_test_split(dfz,dfy,test_size=0.3,random_state=42)

# Normalización de los datos para poder trabajar mejor con mis variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Normalización de los datos para poder trabajar mejor con mis variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Aplico método de PCA
pca = PCA()
X_train = pca.fit_transform(X_train) #entrena y transforma X_train
X_test = pca.transform(X_test) # transforma X_test

# Nuevo análisis de la varianza
explained_variance = pca.explained_variance_ratio_
explained_variance

In [ ]:
# Para tener un acierto de variavilidad de aproximadamente 90% me quedo con 6
# componentes principales

pca = PCA(n_components=6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Creación del modelo
lr = LinearRegression()
# Ajustar el modelo con X_train y y_train
lr.fit(X_train,y_train)
# Predección con X_test
y_pred = lr.predict(X_test)

In [ ]:
#Cálculo de métricas

# Cálculo del Mean Absolute Error (MAE) entre el valor real y y el valor predicho
mae = mean_absolute_error(y_test,y_pred)
mae = round(mae, 3)
print("_ El Mean Absolute Error (MAE) del modelo es",mae)

# Cálculo del Median Absolute Error (Med AE) entre el valor real y y el valor predicho
med_ae = median_absolute_error(y_test,y_pred)
med_ae = round(med_ae, 3)
print("_ El Median Absolute Error (Med AE) del modelo es", med_ae)

# Cálculo del Relative Absolute Error (RAE)
mean_y_test = np.mean(y_test)
rae = mean_absolute_error(y_test,y_pred)/mean_y_test
rae = round(rae, 3)
print("_ El Relative Absolute Error (RAE) del modelo es", rae)

# Cálculo de la raíz cuadrada error cuadrático medio (RMSE) entre el valor real y y el valor predicho
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
rmse = round(rmse, 3)
print("_ El Root Mean Square Error (RMSE) del modelo es",rmse)

# R2 entre y_test y el y predicho
r2 = r2_score(y_test,y_pred)
r2 = round(r2, 3)
print("_ El coeficiente de determinación (R2) del modelo es ",r2)

# LOOCV - regresión
Se opta utulizar el agregado de la variable sintética "var_sint" ya que en el
trabajo previo su incorporación mejoró notablemente las métricas demostrando
ser un mejor dataset

In [ ]:
# Separación en X e y
data= dfz.values
X, y = data[:, :-2], data[:, -2]
print(X.shape, y.shape)

# Creación del procedimiento LOOCV
cv = LeaveOneOut()

# creación el modelo
model = RandomForestRegressor(random_state=42, n_estimators=10,max_depth=4)

In [ ]:
### --- TIEMPO APROXIMADO DE EJECUCIÓN: 19 MINUTOS --- ###

# Se crean las funciones para poder obtener las métricas MSE y R2 que finalmente
# permitirán una justa comparación con el siguiente modelo (Validación simple)
def rmse_scorer(estimador, X, y):
    y_pred = estimador.predict(X)
    return mean_squared_error(y, y_pred)

def r2_scorer(estimador, X, y):
    y_pred = estimador.predict(X)
    return r2_score(y, y_pred)


# Evaluación del modelo (criterio de comparacion con R2)
# Decidí ajustar cv=5 ya que con el propuesto en clase cv = LeaveOneOut()
# R2_scores arrojaba un error computacional
R2_scores = cross_val_score(model, X, y, scoring=r2_scorer, cv=5, verbose=1)
# Converción a positivos
R2_scores = abs(R2_scores)


# Evaluación del modelo (criterio de comparacion con MSE)
MSE_scores = cross_val_score(model, X, y, scoring=rmse_scorer, cv=cv, verbose=1)
# Converción a positivos
MSE_scores = abs(MSE_scores)


# Evaluación del modelo (criterio de comparacion MAE)
MAE = make_scorer(mean_absolute_error)
scores = cross_val_score(model, X, y, scoring=MAE, cv=cv,verbose=1) # se guardan las métricas
# Converción a positivos
scores = abs(scores)

In [ ]:
# Reporte de la performance
print('MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('MSE: %.3f (%.3f)' % (np.mean(MSE_scores), np.std(MSE_scores)))
print('R2: %.3f (%.3f)' % (np.mean(R2_scores), np.std(R2_scores)))

In [ ]:
# Separación de train y test
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, shuffle=True)
print(X_train.shape, X_test.shape)

In [ ]:
# Modelo
model =RandomForestRegressor(random_state=42, n_estimators=10,max_depth=4)
# Ajuste
model.fit(X_train,y_train)

In [ ]:
predicciones= model.predict(X_test)
# Validacion simple
print('MSE: ',mean_squared_error(y_true= y_test, y_pred= predicciones))
print('MAE: ',mean_absolute_error(y_true= y_test, y_pred= predicciones))
print('R2: ',r2_score(y_true= y_test, y_pred= predicciones))

Con base en los resultados actuales de las métricas, es difícil determinar de manera definitiva cuál método de validación produce un mejor modelo.

Las diferencias en las métricas son pequeñas y podrían estar influenciadas por diversos factores como el tamaño del conjunto de datos, la complejidad del modelo y la sensibilidad al sobreajuste.